# Explore here

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import *
from imblearn.metrics import specificity_score

from pickle import dump
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import *

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv")


In [6]:
X = df.drop(['Outcome'], axis= 1)
y = df['Outcome']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 42 )

## Primer modelo ( Decission Tree)

In [7]:
decission_tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=5 , min_samples_leaf=1, min_samples_split=10)
decission_tree_model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=10)

In [8]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [23]:
y_pred_test = decission_tree_model.predict(X_test)
y_pred_train = decission_tree_model.predict(X_train)

get_metrics(y_train, y_test,y_pred_train, y_pred_test)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.819218,0.737589,0.798866,0.742857,0.732394,0.865337
Test,0.785714,0.697248,0.764646,0.703704,0.690909,0.838384
Diferencia,0.033504,0.040341,0.034219,0.039153,0.041485,0.026953


## Segundo modelo (Random Forest)

In [10]:
RF_model = RandomForestClassifier(criterion = 'gini',max_depth=4, min_samples_split=5, max_features=None, bootstrap=True, random_state = 42)
RF_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, max_features=None, min_samples_split=5,
                       random_state=42)

In [11]:
y_pred_test = RF_model.predict(X_test)
y_pred_train = RF_model.predict(X_train)

get_metrics(y_train, y_test,y_pred_train, y_pred_test)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.843648,0.752577,0.806563,0.834286,0.685446,0.927681
Test,0.772727,0.678899,0.750505,0.685185,0.672727,0.828283
Diferencia,0.070921,0.073678,0.056058,0.149101,0.012719,0.099398


## Tercer modelo (Boosting)

In [38]:
model = XGBClassifier(random_state = 42, learning_rate = 0.009, max_depth=4 )
model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)


print(get_metrics(y_train, y_test,y_pred_train, y_pred_test))

            Accuracy        F1       AUC  Precision    Recall  Specificity
Train       0.799674  0.639296  0.732178   0.851562  0.511737     0.952618
Test        0.792208  0.659574  0.741414   0.794872  0.563636     0.919192
Diferencia  0.007466 -0.020278 -0.009236   0.056691 -0.051899     0.033427


In [54]:
hyperparams = { 'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    #'subsample': [0.8, 0.9, 1.0],
    #'colsample_bytree': [0.8, 0.9, 1.0],
    #'gamma': [0, 0.1, 0.2],
    #'scale_pos_weight': [1, 2, 3]
    }

In [55]:
random_search = RandomizedSearchCV(model, param_distributions=hyperparams, n_iter=10, scoring='accuracy', cv=3)
random_search.fit(X_train, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'